In [206]:
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
from sklearn.datasets import load_diabetes
from sklearn.metrics import mean_squared_error




In [98]:
source = load_diabetes()
X = source.data
y = source.target

## GLM Poisson from Stratch

Idea : Write the negative likelihood and use gradient descend to find the optimal value of beta

$$ \frac{\partial L(\beta)}{\partial \beta} = X^T(y-\hat y)$$
$$ \hat y = \exp(X^T\beta)$$

In [24]:
def standard_scaler(x):
    ''' 
    Standartize our feature
    '''
    return (x-x.mean())/x.std()

In [258]:
class regression_poisson:
    def __init__(self, lr=1e-6,n_iter=100000, add_intercept=True, standard =False) :
        self.lr=lr
        self.add_intercept = add_intercept
        self.standard = standard
        self.n_iter = n_iter
        self.beta =  None
        self.mean = None
        self.std =  None
    def fit(self,X,y):
        if self.standard:
            self.mean = X.mean()
            self.std = X.std()
            X = standard_scaler(X)

        if self.add_intercept:
            ones = np.ones((len(X),1))
            X = np.append(ones, X,axis=1)

        beta_hat = np.zeros(X.shape[1])
        for _ in range(self.n_iter):
            y_hat = np.exp(np.dot(X,beta_hat))
            grad = np.dot(X.T,y_hat-y)
            beta_hat+= -self.lr*grad
        self.beta = beta_hat
    def predict(self,X):

        assert self.beta is not None ,'not fitted yet'
        if self.standard:
            X = (X - self.mean) /self.std
        if self.add_intercept:
            ones = np.ones((len(X),1))
            X = np.append(ones, X,axis=1)
        return np.exp(np.dot(X,self.beta))

In [261]:
model= regression_poisson()
model.fit(X,y)
y_predict = model.predict(X)
mean_squared_error(y, y_predict, squared=False)

53.983234599101124

In [253]:
model.beta

array([ 4.96051543,  0.05755439, -1.52591447,  3.16076947,  2.08204279,
       -1.22450743,  0.83003618, -1.91437524, -0.29900342,  3.53782574,
        0.36202108])

## Glm Poisson with statsmodel


In [256]:
X_=sm.add_constant(X)

In [257]:
glm_poisson = sm.GLM(y,X_, family=sm.families.Poisson())
glm_poisson=glm_poisson.fit()
glm_poisson.summary()



<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                  442
Model:                            GLM   Df Residuals:                      431
Model Family:                 Poisson   Df Model:                           10
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -5718.6
Date:                Sat, 23 Dec 2023   Deviance:                       8466.7
Time:                        16:59:34   Pearson chi2:                 8.33e+03
No. Iterations:                     5   Pseudo R-squ. (CS):              1.000
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.9570      0.004   1203.231      0.000       4.949       4.965
x1             0.0198      0.091      0.217      0.828      -0.158       0.198
x2            -1.5894      0.090    -17.586      0.000      -1.767      -1.412
x3             2.9769      0.093     31.983      0.000       2.794       3.159
x4             2.0816      0.095     21.998      0.000       1.896       2.267
x5            -8.9444      0.615    -14.540      0.000     -10.150      -7.739
x6             7.1555      0.494     14.480      0.000       6.187       8.124
x7             1.2613      0.330      3.819      0.000       0.614       1.909
x8             0.1761      0.237      0.744      0.457      -0.288       0.640
x9             6.9012      0.275     25.055      0.000       6.361       7.441
x10            0.3594      0.098      3.678      0.000       0.168       0.551
==============================================================================
"""

In [249]:
y_predict = glm_poisson.predict(X_)
mean_squared_error(y, y_predict, squared=False)

53.14502666105215

In conclusion , the method method one from strach and one with statsmodel give approximately the same result